In [4]:
from google.cloud import vision
from google.cloud import storage
from PIL import Image
import numpy as np
import os, sys
from helper import *
import csv  
from io import BytesIO

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"/Users/jordan/Desktop/Guppies_Home/guppies-test-f7e2b73324f8.json"

### Read all files from the google bucket. Can cycle through the files in this list and write results to the csv file.

In [55]:
all_files = ListAvaliableFiles("guppy_images")

Files have been read.


### Opens a file from the google bucket, crops it, reads it and corrects the output.

In [8]:
file = '1301/Caigual/21A/101NCD60/DSC_1052.JPG'
image = RetreiveImage(file)
cropped_image = CroppedImage(image)
output_string, word_confidences = ReadImage(cropped_image)
label = FindErrors(output_string, verbose=False)
label


Image has been read from google bucket.

Image has been cropped.

Output: FCA-ZYBY-01/21/13 
Confidence: 0.60731311116466


'FCA-2Y3Y-01/21/13'

### Write filename, initial prediction and corrected prediction to csv file for a set of file names.

In [4]:
with open('../Data/predictions.csv', 'w') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(['filename', 'prediction', 'corrected'])

    for i in range(len(all_files)):    
        print(f'{i}/{len(all_files)}',end='\r')
        # write the data
        prediction = ReadImage(all_files[i], verbose=False)[0]

        corrected_prediction = FindErrors(prediction)

        writer.writerow([all_files[i], prediction, corrected_prediction])